In [63]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [64]:
import geopy, os
import pandas as pd
from geopy.geocoders import GoogleV3

In [65]:
API_FILE = "api.txt"
DATA_FILE = "COMERCIO.TXT"
LOC_FILE = "localized_addresses.txt"

In [66]:
def read_api_keys():
    with open(API_FILE, 'r') as f:
        yield f.read().rstrip()
        
def read_saved_data():
    if os.path.exists(LOC_FILE):
        return pd.read_csv(LOC_FILE,index_col=0)
    else:
        return pd.read_fwf(DATA_FILE, encoding='ISO-8859-1')

def geolocate(addr, geolocators):
    try:
        random_locator = np.random.choice(geolocators)
        return random_locator.geocode("Porto Alegre %s" % addr)[1]
    except TypeError:
        print("Não pode geolocalizar: %s" % x)
        return (None, None)
    except NameError:
        print("Timeout %s" % addr)
        return (None, None)
    except geopy.exc.GeocoderTimedOut:
        print("Timeout %s" % addr)
        return (None, None)

data = read_saved_data()

if not "Latitude" in data:
    data = pd.DataFrame(data["Endereço"].dropna())
    data["Latitude"] = np.NaN
    data["Longitude"] = np.NaN
    
geolocators = [ GoogleV3(x) for x in read_api_keys() ]

locate_dict = data.drop_duplicates().to_dict(orient='records')

print("Localized: %s" % (data[data["Latitude"].notnull()].size/data.size))
print("Updating known localizations...")
try:
    for d in locate_dict:
        if np.isnan(d["Latitude"]):
            result = geolocate(d["Endereço"], geolocators)
            d["Latitude"], d["Longitude"] = result
            
except Exception as e:
    print(e)  
    
data.update(pd.DataFrame(locate_dict))
print("New localized amount: %s" % (data[data["Latitude"].notnull()].size/data.size))


data[["Endereço","Latitude","Longitude"]].to_csv(LOC_FILE)

Localized: 0.0287760528347
Updating known localizations...
The given key has gone over the requests limit in the 24 hour period or has submitted too many requests in too short a period of time.
New localized amount: 0.064470938045


In [67]:
import folium
from folium.plugins import HeatMap

# HeatMap Visualization

In [68]:
m = folium.Map((-30,-51), zoom_start=9, tiles='stamentoner')
points = data[["Latitude","Longitude"]][data["Latitude"].notnull()]
heat = HeatMap(points.as_matrix())
m.add_children(heat)

# Cluster Visualization

In [69]:
m2 = folium.Map((-30,-51), zoom_start=9)
cluster = folium.MarkerCluster().add_to(m2)
for point in points.as_matrix():
    folium.Marker(point).add_to(cluster)
m2